- make sure resetting works as expected
- tests for resetting
- what things to expose in the object?
- what things to expose in the top level interface?
- checking that the prox is correct?
- hide scswork
- just expose the appropriate settings
- explain how settings can be passed down
- show that it knows its own zero element
- can take in an empty dict

# `scsprox` Demo
`scsprox` creates fast proximal operators from CVXPY `Problem` objects.

`scsprox` uses CVXPY to form the proximal operator problem and translate it to the SCS conic input format.
This translation is performed **only once** during the `Prox` object initialization to save time.
`scsprox` uses CySCS for matrix-factorization-caching and warm-starting to reduce solve times
over many repeated solves, as occurs, for example, within the alternating-direction method of multipliers (ADMM)
algorithm.

- for the demo, first initialize a simple CVXPY problem

In [1]:
import numpy as np
import cvxpy as cvx
from scsprox import Prox

m, n = 200, 100

A = np.random.randn(m,n)
b = np.random.randn(m)
x = cvx.Variable(n)

prob = cvx.Problem(cvx.Minimize(cvx.norm(A*x-b)))
prob.solve()

x_true = np.array(x.value).flatten()
#x_true

## form the `Prox` object
- create the prox object by passing in the CVXPY problem, along with a dict, `prox_vars`, of the proximal variables
- during initialization, the `Prox` object forms a CySCS `Workspace`, which computes and stores the SCS factorization (which only needs to be computed once)
- the `Prox` object accepts arbitrary CVXPY problems and any dict of related CVXPY variables to form the prox

In [2]:
prox_vars = {'x': x}
prox = Prox(prob, prox_vars)

## evaluate the prox
- evaluate the prox, noting that SCS **doesn't** initialize, because the factorization has been cached
- note that this first call to `prox.do()` takes 40 iterations
- the prox operator is evaluted on `x0`, a dictionary of variable names and values (matching the names and variable sizes in `prox_vars`)
- within SCS, the `A` matrix doesn't change between prox calls, but the `b` and `c` vectors do change
    - `Prox` automatically restuffs the values in `b` and `c` based on the data in `x0`
    - restuffing is just an **indexing** operation, and **avoids the overhead** of CVXPY entirely

In [3]:
x0 = {'x': np.zeros(n)}
rho = 1.0
x1 = prox.do(x0, rho, verbose=True)

SCS using variable warm-starting
----------------------------------------------------------------------------
 Iter | pri res | dua res | rel gap | pri obj | dua obj | kap/tau | time (s)
----------------------------------------------------------------------------
     0| 1.70e+00  6.16e+00  9.86e-01 -2.06e+01  4.81e+01  0.00e+00  2.36e-03 
    40| 1.83e-07  2.43e-06  2.55e-07  1.06e+01  1.06e+01  2.16e-15  7.35e-03 
----------------------------------------------------------------------------
Status: Solved
Timing: Solve time: 7.36e-03s
	Lin-sys: nnz in L factor: 30408, avg solve time: 1.17e-04s
	Cones: avg projection time: 7.50e-07s
----------------------------------------------------------------------------
Error metrics:
dist(s, K) = 1.1102e-16, dist(y, K*) = 0.0000e+00, s'y/|s||y| = -2.1643e-16
|Ax + s - b|_2 / (1 + |b|_2) = 1.8296e-07
|A'y + c|_2 / (1 + |c|_2) = 2.4284e-06
|c'x + b'y| / (1 + |c'x| + |b'y|) = 2.5491e-07
---------------------------------------------------------------

In [11]:
prox.info

{'iter': 0,
 'setup_time': 0.005640603,
 'solve_time': 0.0021005859999999998,
 'status': 'Solved'}

## automatic warm-starting
- if we call `prox.do()` again, we can take advantage of warm-starting
- the `Prox` object caches the last SCS solution, and uses it to warm-start the next SCS solve
- with the same `x0` as the previous call, SCS does **0** iterations, because the warm-start solution from the previous iteration is already within the solver tolerance

In [6]:
x0 = {'x': np.zeros(n)}
rho = 1.0
x1 = prox.do(x0, rho)

SCS using variable warm-starting
----------------------------------------------------------------------------
 Iter | pri res | dua res | rel gap | pri obj | dua obj | kap/tau | time (s)
----------------------------------------------------------------------------
     0| 2.00e-07  1.89e-06  1.10e-07  1.06e+01  1.06e+01  0.00e+00  2.11e-03 
----------------------------------------------------------------------------
Status: Solved
Timing: Solve time: 2.12e-03s
	Lin-sys: nnz in L factor: 30408, avg solve time: 3.06e-04s
	Cones: avg projection time: 2.52e-06s
----------------------------------------------------------------------------
Error metrics:
dist(s, K) = 0.0000e+00, dist(y, K*) = 0.0000e+00, s'y/|s||y| = 5.1352e-16
|Ax + s - b|_2 / (1 + |b|_2) = 2.0002e-07
|A'y + c|_2 / (1 + |c|_2) = 1.8942e-06
|c'x + b'y| / (1 + |c'x| + |b'y|) = 1.0985e-07
----------------------------------------------------------------------------
c'x = 10.6029, -b'y = 10.6029


## warm-starting 2
- of course, we usually won't try to compute the prox on exactly the same value, but instead, a slight perturbation of that value
- warm-starting still helps in this case, and still works automatically
- we call the prox on `x1`, the output of the first prox computation
- SCS is warm-started from the previous solution, which will tend to reduce the number of iterations needed

In [7]:
rho = 1.0
x2 = prox.do(x1, rho)

SCS using variable warm-starting
----------------------------------------------------------------------------
 Iter | pri res | dua res | rel gap | pri obj | dua obj | kap/tau | time (s)
----------------------------------------------------------------------------
     0| 6.45e-02  1.08e+00  8.74e-02  1.01e+01  8.43e+00  0.00e+00  2.36e-03 
    20| 9.01e-06  2.04e-04  7.32e-06  1.02e+01  1.02e+01  3.28e-15  4.35e-03 
----------------------------------------------------------------------------
Status: Solved
Timing: Solve time: 4.37e-03s
	Lin-sys: nnz in L factor: 30408, avg solve time: 9.85e-05s
	Cones: avg projection time: 7.69e-07s
----------------------------------------------------------------------------
Error metrics:
dist(s, K) = 1.1102e-16, dist(y, K*) = 0.0000e+00, s'y/|s||y| = 4.6260e-16
|Ax + s - b|_2 / (1 + |b|_2) = 9.0094e-06
|A'y + c|_2 / (1 + |c|_2) = 2.0359e-04
|c'x + b'y| / (1 + |c'x| + |b'y|) = 7.3157e-06
----------------------------------------------------------------

## proximal iteration
- as an example application, we can solve the original CVXPY problem through proximal iteration

In [8]:
for i in range(20):
    x0 = prox.do(x0, rho, verbose=False)

- note that, after several iterations, proximal iteration converges, and the SCS solver finishes in **0** iterations

In [9]:
x0 = prox.do(x0, rho, verbose=True)

SCS using variable warm-starting
----------------------------------------------------------------------------
 Iter | pri res | dua res | rel gap | pri obj | dua obj | kap/tau | time (s)
----------------------------------------------------------------------------
     0| 3.54e-06  1.40e-05  5.37e-08  1.02e+01  1.02e+01  0.00e+00  2.08e-03 
----------------------------------------------------------------------------
Status: Solved
Timing: Solve time: 2.10e-03s
	Lin-sys: nnz in L factor: 30408, avg solve time: 1.97e-02s
	Cones: avg projection time: 1.24e-04s
----------------------------------------------------------------------------
Error metrics:
dist(s, K) = 0.0000e+00, dist(y, K*) = 2.2204e-16, s'y/|s||y| = 5.8841e-17
|Ax + s - b|_2 / (1 + |b|_2) = 3.5369e-06
|A'y + c|_2 / (1 + |c|_2) = 1.3954e-05
|c'x + b'y| / (1 + |c'x| + |b'y|) = 5.3663e-08
----------------------------------------------------------------------------
c'x = 10.2119, -b'y = 10.2119


- we can do the same thing within an ADMM iteration:
    - set the tolerance to, say, 1e-5
    - set the maximum number of SCS iterations (per prox) to, say, 10 or 100
    - this results in ADMM with approximate proximal evaluations (only a few SCS iterations), but which eventually converges (via warm-starting) to the desired tolerance of 1e-5